In [1395]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('dark_background')

In [1396]:
df = pd.read_csv('data/house_price/train.csv', index_col=0)
val_df = pd.read_csv('data/house_price/test.csv', index_col=0)

# Data Processing

## Feature Engineering

In [1397]:
print(val_df.shape)


def feature_engineering_log1p(df, col_name):
    df = df.copy()
    df[col_name] = df[col_name].fillna(df[col_name].median())
    df[col_name + "_log1p"] = np.log1p(df[col_name].fillna(df[col_name].median()))
    return df


df = feature_engineering_log1p(df, 'MasVnrArea')
val_df = feature_engineering_log1p(val_df, 'MasVnrArea')
print(val_df.shape)

(1459, 79)
(1459, 80)


In [1398]:
def feature_engineering_fillna_random(df, col_name):
    df = df.copy()
    df[col_name] = df[col_name].to_frame().applymap(
        lambda l:
        l if not np.isnan(l)
        else np.random.choice(df[col_name].dropna()))
    return df


df = feature_engineering_fillna_random(df, "GarageYrBlt")
val_df = feature_engineering_fillna_random(val_df, "GarageYrBlt")

In [1399]:
print(val_df.shape)

(1459, 80)


## Filling Na and Categorical One Hot Encoding

In [1400]:
pd.get_dummies?

In [1401]:
cols = ["MSZoning", "Street", "Alley", "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope", "Condition1",
        "Condition2",
        "Neighborhood", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType",
        "ExterQual",
        "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating",
        "HeatingQC",
        "CentralAir", "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageFinish",
        "GarageQual", "GarageCond",
        "PavedDrive", "PoolQC", "Fence", "MiscFeature", "SaleType", "SaleCondition"]

fill_na_columns = ['BsmtFinSF1',
                   'BsmtFinSF2',
                   'BsmtUnfSF',
                   'TotalBsmtSF',
                   'BsmtFullBath',
                   'BsmtHalfBath',
                   'GarageCars',
                   'GarageArea',
                   'LotFrontage'
                   ]

fill_none_columns = ['Alley',
                     'BsmtQual',
                     'BsmtCond',
                     'BsmtExposure',
                     'PoolQC',
                     'Fence',
                     'MiscFeature'
                     ]

In [1402]:
def format_data(df, columns, fill_na_columns, fill_none_columns):
    for fill_na_column in fill_na_columns:
        mean = pd.to_numeric(df[fill_na_column].mean())
        df[fill_na_column].fillna(mean, inplace=True)

    for fill_none_column in fill_none_columns:
        df[fill_none_column].fillna('Not precised', inplace=True)

    for column in columns:
        df = pd.concat([df, pd.get_dummies(data=df[column], prefix=column)], axis=1)

    df.drop(columns=columns, inplace=True, errors='ignore')
    return df

# this won't work because test data has different categories and the resulting df will be of different shape than the training one -> the resolution was to strongly modify test data

In [1403]:
# df.columns
# val_df.columns
# horizontal_stack = pd.concat([val_df, df], axis=1)
val_df.shape
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,MasVnrArea_log1p
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,5.283204
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,0.000000
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,5.093750
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,0.000000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,5.860786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000,0.000000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000,4.787492
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500,0.000000


In [1404]:
predictor_cols = ['LotArea', 'OverallQual', 'YearBuilt', 'TotRmsAbvGrd']

df = format_data(df, cols, fill_na_columns, fill_none_columns)
val_df = format_data(val_df, cols, fill_na_columns, fill_none_columns)

y = df.SalePrice
#X = df[predictor_cols]
X = df.drop('SalePrice', axis=1)

#val_df = val_df[predictor_cols]
val_df.shape

(1459, 278)

In [1405]:
# VarianceThreshold?
# from sklearn.feature_selection import VarianceThreshold
# sel = VarianceThreshold(threshold=(0.01))
# sel.fit(X)
# X = X[X.columns[sel.get_support(indices=True)]]
horizontal_stack = pd.concat([val_df, df], axis=0)
horizontal_stack.drop(horizontal_stack.tail(1460).index,
                      inplace=True)
val_df = horizontal_stack
val_df = val_df.drop('SalePrice', axis=1)
val_df

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MasVnrArea_log1p,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Not precised,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Not precised,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Not precised,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_Not precised,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Gd,PoolQC_Not prec

In [1406]:
missing_columns = ['Utilities_NoSeWa',
                   'Condition2_RRAe',
                   'Condition2_RRAn',
                   'Condition2_RRNn',
                   'HouseStyle_2.5Fin',
                   'RoofMatl_ClyTile',
                   'RoofMatl_Membran',
                   'RoofMatl_Metal',
                   'RoofMatl_Roll',
                   'Exterior1st_ImStucc',
                   'Exterior1st_Stone',
                   'Exterior2nd_Other',
                   'Heating_Floor',
                   'Heating_OthW',
                   'Electrical_Mix',
                   'GarageQual_Ex',
                   'PoolQC_Fa',
                   'MiscFeature_TenC']

for c in missing_columns:
    val_df[c].fillna(0, inplace=True)

In [1407]:
val_df.isna().any()

MSSubClass          False
LotFrontage         False
LotArea             False
OverallQual         False
OverallCond         False
                    ...  
Heating_OthW        False
Electrical_Mix      False
GarageQual_Ex       False
PoolQC_Fa           False
MiscFeature_TenC    False
Length: 296, dtype: bool

# Modeling

## Decision Tree Regressor

In [1408]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

dec_tree = DecisionTreeRegressor()
param_space = {
    # 'min_samples_leaf': range(1, 10),
    # 'max_depth': range(1, 15)
    #"splitter": ["best", "random"],
    "max_depth": range(1, 13),
    "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"min_weight_fraction_leaf": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    #"max_features":["auto","log2","sqrt",None],
    #"max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90]
}
gs = GridSearchCV(dec_tree, param_space, verbose=1, cv=5, return_train_score=True)
gs.fit(X, y)
results = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score').iloc[1, 4:]

Fitting 5 folds for each of 120 candidates, totalling 600 fits


In [1409]:
results.mean_train_score, results.mean_test_score, results.params

(0.8966511728304047,
 0.7803718565670319,
 {'max_depth': 12, 'min_samples_leaf': 8})

## KNN

In [1410]:
from sklearn.neighbors import KNeighborsRegressor

k_neighbors = KNeighborsRegressor()
param_space = {
    'n_neighbors': range(1, 11),
}
gs = GridSearchCV(k_neighbors, param_space, verbose=1, cv=5, return_train_score=True)
gs.fit(X, y)
results = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score').iloc[1, 4:]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [1411]:
results.mean_train_score, results.mean_test_score, results.params

(0.7160875536823653, 0.6530130141783164, {'n_neighbors': 10})

## SVC

In [1412]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.svm import SVR
#
# svr = SVR()
# param_space = [
#     {
#         "kernel": ["linear"],
#         "C": [1, 10, 100,],
#     },
#     {
#         "kernel": ["rbf"],
#         "C": [10, 100],
#         "gamma": [0.001, 0.0001],
#     },
# ]
# gs = GridSearchCV(svr, param_space, verbose=1, cv=5, return_train_score=True)
# gs.fit(X, y)
# results = pd.DataFrame(gs.cv_results_).sort_values(by="rank_test_score").iloc[1, 4:]

In [1413]:
# results.mean_train_score, results.mean_test_score, results.params

# Predict the submission

In [1414]:
val_df.isna().any()

MSSubClass          False
LotFrontage         False
LotArea             False
OverallQual         False
OverallCond         False
                    ...  
Heating_OthW        False
Electrical_Mix      False
GarageQual_Ex       False
PoolQC_Fa           False
MiscFeature_TenC    False
Length: 296, dtype: bool

In [1415]:
predicted_df = gs.predict(val_df)
submission_df = pd.DataFrame({'SalePrice': val_df.index}, index=val_df.index)
submission_df['SalePrice'] = predicted_df
submission_df.sample(3, random_state=7)

submission_df.to_csv('submission.csv')

C:\Users\admin\anaconda3\envs\ds_env\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


# Do Domu

- ~~KNN~~
- Feature Engineering zrobić funkcje i w pętli przeiterować przez wszystkie kolumny

Do Przeczytania:
- CNN [Blog](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53)
- DNN [Video](https://www.youtube.com/watch?v=aircAruvnKk)
- CNN [Video](https://www.youtube.com/watch?v=YRhxdVk_sIs)

# Następna lekcja

problem MNIST classification - ręcznie pisane cyfry 2 rozwiązania:

    - DNN - sieć gęsta
    - CNN - sieć splotowa (ang. convolutional neural network)